In [ ]:
import pandas as pd
import os
import frameprocessing
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

pd.set_option("display.max_columns", None)

In [ ]:
path_of_the_directory= './excel/'
data = pd.DataFrame()
for filename in os.listdir(path_of_the_directory):
    f = os.path.join(path_of_the_directory,filename)
    if os.path.isfile(f):
        if filename != '.DS_Store':
            temp = pd.read_excel(f, keep_default_na=False)
            temp = temp[2:]
            data = pd.concat([data, temp])

data

#BIG CHANGES:
#REMOVE TEAM ALL TOGETHER
#ONLY DO HOME/AWAY

In [ ]:

#data = data.iloc[:,0:8]
#df_new = df.iloc[:, 0:3]
#filtrera allsvenskan 
data = data.loc[data["Competition"]=="Sweden. Allsvenskan"]

data['Home'] = data.apply(lambda x: frameprocessing.getHomeTeam(x['Match']), axis=1)
data['Away'] = data.apply(lambda x: frameprocessing.getAwayTeam(x['Match']), axis=1)
data["Result"] = data.apply(lambda x: frameprocessing.getResult(x['Match']), axis=1)
data["Date"] = data['Date'].apply(lambda x: x.replace("-","/")).apply(lambda x: frameprocessing.dateSwitcher(x))

home = data.pop("Home")
away = data.pop("Away")
result = data.pop("Result")
data.insert(1, "Home", home)
data.insert(2,"Away", away)
data.insert(3,"Result",result)

#drop onödigt(scheme,duration,competition, match)
data.drop(["Match"],axis=1, inplace=True)
data.drop(["Scheme","Competition","Duration"],axis=1,inplace=True)

data.reset_index(inplace=True, drop=True)

team_1,team_2 = frameprocessing.splitData(data)

team_2.drop(["Date","Home"],axis=1,inplace=True)
team_2.columns = frameprocessing.get_away_columns(team_2)

team_1.reset_index(inplace=True, drop=True)
team_2.reset_index(inplace=True, drop=True)
concat_df = pd.concat([team_1,team_2],axis="columns")

concat_df = concat_df.loc[concat_df["Team"] != "Östersunds FK"]
concat_df = concat_df.loc[concat_df["Team"] != "Örebro"]
concat_df = concat_df.loc[concat_df["Team"] != "Falkenberg"]
concat_df = concat_df.loc[concat_df["away Team"] != "Östersunds FK"]
concat_df = concat_df.loc[concat_df["away Team"] != "Örebro"]
concat_df = concat_df.loc[concat_df["away Team"] != "Falkenberg"]

#gör av med duplicates
#concat_df["unique_string"] = concat_df.apply(lambda x: str(x["Date"]) + str(x["Home"]),axis=1)
#concat_df = concat_df.drop_duplicates(subset = ["unique_string"]).drop(["unique_string"],axis=1)

#spara home 
concat_df


In [ ]:
data_avg = data.drop(['Date','Home'], axis=1).groupby('Team').mean()
data_avg = data_avg.reset_index()
data_avg

In [ ]:
concat_df["Result"] = concat_df.apply(lambda x: 1 if x["Goals"] > x["away Goals"] else 0 if x["Goals"] == x["away Goals"] else 2,axis=1)
concat_df = frameprocessing.one_hot(concat_df)
concat_df.reset_index(inplace=True,drop=True)
concat_df

# transform dataframe to average

In [ ]:

asd = frameprocessing.matches_to_dataframe(concat_df, data_avg)
asd

In [ ]:
X = asd.drop(["Result"], axis=1)
y = asd["Result"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_test_copy = X_test.copy(deep=True)

X_test

In [ ]:
encoder = LabelEncoder()

encoder.fit(y)
encoded_y_tjark = encoder.transform(y)
dummy = np_utils.to_categorical(encoded_y_tjark)

encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
dummy_y = np_utils.to_categorical(encoded_Y)

encoder.fit(y_test)
encoded_Y_test = encoder.transform(y_test)
dummy_y_test = np_utils.to_categorical(encoded_Y_test)

In [ ]:
X_train.drop(["Date", "Team", "away Team"], inplace=True, axis=1)
X_test.drop(["Date", "Team", "away Team"], inplace=True, axis=1)

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

model = Sequential()
model.add(Dense(len(X_train[0]), input_dim = len(X_train[0]), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, dummy_y, epochs=100, batch_size=100)

In [ ]:
y_pred = model.predict(X_test)

res = pd.DataFrame(y_pred, columns=["Draw %", "Win %", "Loss %"])
X_test_copy.reset_index(inplace=True,drop=True)
res = pd.concat([res, X_test_copy[["Date", "Team", "away Team", "Home"]]], axis="columns")
odds = pd.read_csv("SWE.csv")
odds = odds[["Date", "Home","AvgH", "AvgD", "AvgA"]]

#Modify name
res["Home"] = res.apply(lambda x: frameprocessing.transformTeam(x["Team"]) if x["Home"] == 1 else frameprocessing.transformTeam(x["away Team"]), axis=1)

res = res.merge(odds, how= "inner", left_on=["Date", "Home"],right_on=["Date", "Home"])

date = res.pop("Date")
team = res.pop("Team")
oppos = res.pop("away Team")
home = res.pop("Home")

res.insert(0, "Date", date)
res.insert(1, "Team", team)
res.insert(2, "away Team", oppos)
res.insert(3, "Home", home)

res["AvgH %"] = np.around(100/res["AvgH"],2)
res["AvgD %"] = np.around(100/res["AvgD"],2)
res["AvgA %"] = np.around(100/res["AvgA"],2)

res["Draw %"] = np.around(res["Draw %"] * 100,2)
res["Win %"] = np.around(res["Win %"] * 100,2)
res["Loss %"] = res["Loss %"].round(2)*100

res = res.round(2)
res

# we are fuccccccccccccccccccked ... avgH => is odds for home,

# lambda funktion kanske kan ändra så att våra predictions passar in på oddsen
# eller tvärtom

# our odds are for "Team" win

In [ ]:
#david har koden